In [2]:
import numpy as np
import warnings
import sys
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:                                         
    warnings.simplefilter("ignore")
import pickle

In [3]:
def get_data(path, noise=False):
    data = np.load(path)
    if noise == True :
        data = data + np.random.normal(0, 0.001, data.shape)
    return data

In [4]:
X_train = get_data('enron-train-features.pkl').astype(np.uint8)
y_train = get_data('enron-train-labels.pkl').astype(np.int8)
X_test = get_data('enron-test-features.pkl').astype(np.uint8)
y_test = get_data('enron-test-labels.pkl').astype(np.int8)

In [5]:
print("Train_X: ",X_train.shape)
print("Train_Y: ",y_train.shape)
print("Test_X: ",X_test.shape)
print("Test_Y: ",y_test.shape)

Train_X:  (1123, 1001)
Train_Y:  (1123, 53)
Test_X:  (579, 1001)
Test_Y:  (579, 53)


In [6]:
def hamming_accuracy_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

## Chain Model type 1


In [7]:
import keras
from keras.models import Sequential, model_from_json
from keras.layers import Dense,Dropout
from keras.backend.tensorflow_backend import clear_session

Using TensorFlow backend.


In [8]:
def create_c1node(X_feed,y_now):
    '''
    C1 node Architecture:
    attribute:512:256:1 [saperate for each class]
    loss: Binary crossentropy
    '''
    model = Sequential()
    model.add(Dense(512,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='glorot_uniform',name='first'))
    model.add(Dropout(0.4))
    model.add(Dense(256,activation='relu',kernel_initializer='glorot_uniform',name='nretrain1'))
    model.add(Dropout(0.4))
    model.add(Dense(128,activation='relu',kernel_initializer='glorot_uniform',name='nretrain2'))
    model.add(Dropout(0.4))
    model.add(Dense(64,activation='relu',kernel_initializer='glorot_uniform',name='last'))
    model.add(Dropout(0.4))    
    model.add(Dense(y_now.shape[1],activation='sigmoid',kernel_initializer='glorot_uniform',name='output'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

def create_copynode(X_feed,y_now,w_name):
    '''
    C1 node Architecture:
    attribute:512:256:1 [saperate for each class]
    loss: Binary crossentropy
    '''
    model = Sequential()
    model.add(Dense(512,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='glorot_uniform',name='copyfirst'))
    model.add(Dropout(0.4))
    model.add(Dense(256,activation='relu',kernel_initializer='glorot_uniform',trainable=False,name='nretrain1'))
    model.add(Dropout(0.4))
    model.add(Dense(128,activation='relu',kernel_initializer='glorot_uniform',trainable=False,name='nretrain2'))
    model.add(Dropout(0.4))
    model.add(Dense(64,activation='relu',kernel_initializer='glorot_uniform',name='copylast'))
    model.add(Dropout(0.4)) 
    model.add(Dense(y_now.shape[1],activation='sigmoid',kernel_initializer='glorot_uniform',name='copyoutput'))
    model.load_weights(w_name,by_name=True)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [9]:
#Model chain
chain = []
first = 1
#Training
X_feed = X_train.copy()
for i in range(y_train.shape[1]):
    print("Training chain node ",i)
    y_now = y_train[:,[i,]].copy()
    print("Shapes:\n X = {} \n Y = {}".format(X_feed.shape,y_now.shape))
    if first == 1:
        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=20,batch_size=50)
        '''
        json_fl = node.to_json()
        with open('node1.json','w') as f:
            f.write(json_fl)
        '''
        node.save_weights('weights.h5')
        first = 0
    else:
        node = create_copynode(X_feed,y_now,'weights.h5')
        node.fit(X_feed,y_now,epochs=10,batch_size=50)
        
    print("Training of node {} complete\n\n".format(i))  
    #Checking accuracy of the node
    #---
    #Adding node to chain 
    chain.append(node)
    #Updating X_feed
    X_feed = np.append(X_feed,y_now,axis=1)


Training chain node  0
Shapes:
 X = (1123, 1001) 
 Y = (1123, 1)
Epoch 1/20
1123/1123 [==============================] - 16s 14ms/step - loss: 0.3299 - acc: 0.8744
Epoch 2/20
1123/1123 [==============================] - 0s 249us/step - loss: 0.1516 - acc: 0.9840
Epoch 3/20
1123/1123 [==============================] - 0s 353us/step - loss: 0.1073 - acc: 0.9840
Epoch 4/20
1123/1123 [==============================] - 0s 275us/step - loss: 0.1002 - acc: 0.9840
Epoch 5/20
1123/1123 [==============================] - 0s 287us/step - loss: 0.0780 - acc: 0.9840
Epoch 6/20
1123/1123 [==============================] - 0s 267us/step - loss: 0.0735 - acc: 0.9840
Epoch 7/20
1123/1123 [==============================] - 0s 255us/step - loss: 0.0631 - acc: 0.9840
Epoch 8/20
1123/1123 [==============================] - 0s 249us/step - loss: 0.0606 - acc: 0.9840
Epoch 9/20
1123/1123 [==============================] - 0s 258us/step - loss: 0.0551 - acc: 0.9840
Epoch 10/20
1123/1123 [=====================

1123/1123 [==============================] - 0s 242us/step - loss: 0.3974 - acc: 0.8664
Epoch 8/10
1123/1123 [==============================] - 0s 230us/step - loss: 0.3902 - acc: 0.8842
Epoch 9/10
1123/1123 [==============================] - 0s 231us/step - loss: 0.3177 - acc: 0.9047
Epoch 10/10
1123/1123 [==============================] - 0s 249us/step - loss: 0.2971 - acc: 0.9118
Training of node 6 complete


Training chain node  7
Shapes:
 X = (1123, 1008) 
 Y = (1123, 1)
Epoch 1/10
1123/1123 [==============================] - 1s 1ms/step - loss: 0.4683 - acc: 0.8317
Epoch 2/10
1123/1123 [==============================] - 0s 233us/step - loss: 0.1851 - acc: 0.9733
Epoch 3/10
1123/1123 [==============================] - 0s 233us/step - loss: 0.1267 - acc: 0.9733
Epoch 4/10
1123/1123 [==============================] - 0s 261us/step - loss: 0.0902 - acc: 0.9733
Epoch 5/10
1123/1123 [==============================] - 0s 230us/step - loss: 0.0693 - acc: 0.9751
Epoch 6/10
1123/1123 [====

1123/1123 [==============================] - 0s 239us/step - loss: 0.6788 - acc: 0.5663
Epoch 3/10
1123/1123 [==============================] - 0s 224us/step - loss: 0.6265 - acc: 0.6643
Epoch 4/10
1123/1123 [==============================] - 0s 236us/step - loss: 0.5854 - acc: 0.7435
Epoch 5/10
1123/1123 [==============================] - 0s 227us/step - loss: 0.4839 - acc: 0.8139
Epoch 6/10
1123/1123 [==============================] - 0s 233us/step - loss: 0.3968 - acc: 0.8459
Epoch 7/10
1123/1123 [==============================] - 0s 242us/step - loss: 0.3068 - acc: 0.8985
Epoch 8/10
1123/1123 [==============================] - 0s 235us/step - loss: 0.2141 - acc: 0.9341
Epoch 9/10
1123/1123 [==============================] - 0s 238us/step - loss: 0.1850 - acc: 0.9510
Epoch 10/10
1123/1123 [==============================] - 0s 229us/step - loss: 0.1456 - acc: 0.9564
Training of node 14 complete


Training chain node  15
Shapes:
 X = (1123, 1016) 
 Y = (1123, 1)
Epoch 1/10
1123/1123 [

1123/1123 [==============================] - 0s 236us/step - loss: 0.0324 - acc: 0.9920
Epoch 9/10
1123/1123 [==============================] - 0s 235us/step - loss: 0.0266 - acc: 0.9947
Epoch 10/10
1123/1123 [==============================] - 0s 234us/step - loss: 0.0235 - acc: 0.9929
Training of node 21 complete


Training chain node  22
Shapes:
 X = (1123, 1023) 
 Y = (1123, 1)
Epoch 1/10
1123/1123 [==============================] - 2s 2ms/step - loss: 0.7171 - acc: 0.7498
Epoch 2/10
1123/1123 [==============================] - 0s 234us/step - loss: 0.1776 - acc: 0.9644
Epoch 3/10
1123/1123 [==============================] - 0s 238us/step - loss: 0.1309 - acc: 0.9644
Epoch 4/10
1123/1123 [==============================] - 0s 238us/step - loss: 0.0974 - acc: 0.9644
Epoch 5/10
1123/1123 [==============================] - 0s 241us/step - loss: 0.0691 - acc: 0.9644
Epoch 6/10
1123/1123 [==============================] - 0s 236us/step - loss: 0.0556 - acc: 0.9671
Epoch 7/10
1123/1123 [==

1123/1123 [==============================] - 0s 243us/step - loss: 0.1271 - acc: 0.9528
Epoch 3/10
1123/1123 [==============================] - 0s 263us/step - loss: 0.0950 - acc: 0.9724
Epoch 4/10
1123/1123 [==============================] - 0s 256us/step - loss: 0.0815 - acc: 0.9768
Epoch 5/10
1123/1123 [==============================] - 0s 256us/step - loss: 0.0602 - acc: 0.9804
Epoch 6/10
1123/1123 [==============================] - 0s 258us/step - loss: 0.0493 - acc: 0.9858
Epoch 7/10
1123/1123 [==============================] - 0s 263us/step - loss: 0.0505 - acc: 0.9849
Epoch 8/10
1123/1123 [==============================] - 0s 269us/step - loss: 0.0529 - acc: 0.9858
Epoch 9/10
1123/1123 [==============================] - 0s 252us/step - loss: 0.0718 - acc: 0.9822
Epoch 10/10
1123/1123 [==============================] - 0s 247us/step - loss: 0.0664 - acc: 0.9849
Training of node 29 complete


Training chain node  30
Shapes:
 X = (1123, 1031) 
 Y = (1123, 1)
Epoch 1/10
1123/1123 [

1123/1123 [==============================] - 0s 253us/step - loss: 0.0057 - acc: 0.9991
Epoch 9/10
1123/1123 [==============================] - 0s 278us/step - loss: 0.0040 - acc: 0.9991
Epoch 10/10
1123/1123 [==============================] - 0s 272us/step - loss: 0.0033 - acc: 1.0000
Training of node 36 complete


Training chain node  37
Shapes:
 X = (1123, 1038) 
 Y = (1123, 1)
Epoch 1/10
1123/1123 [==============================] - 3s 3ms/step - loss: 0.2627 - acc: 0.9341
Epoch 2/10
1123/1123 [==============================] - 0s 419us/step - loss: 0.1143 - acc: 0.9804
Epoch 3/10
1123/1123 [==============================] - 0s 262us/step - loss: 0.0591 - acc: 0.9822
Epoch 4/10
1123/1123 [==============================] - 0s 295us/step - loss: 0.0400 - acc: 0.9822
Epoch 5/10
1123/1123 [==============================] - 0s 263us/step - loss: 0.0268 - acc: 0.9858
Epoch 6/10
1123/1123 [==============================] - 0s 251us/step - loss: 0.0227 - acc: 0.9902
Epoch 7/10
1123/1123 [==

1123/1123 [==============================] - 0s 250us/step - loss: 0.1482 - acc: 0.9279
Epoch 4/10
1123/1123 [==============================] - 0s 259us/step - loss: 0.1211 - acc: 0.9394
Epoch 5/10
1123/1123 [==============================] - 0s 252us/step - loss: 0.0891 - acc: 0.9510
Epoch 6/10
1123/1123 [==============================] - 0s 255us/step - loss: 0.0735 - acc: 0.9671
Epoch 7/10
1123/1123 [==============================] - 0s 257us/step - loss: 0.0717 - acc: 0.9804
Epoch 8/10
1123/1123 [==============================] - 0s 249us/step - loss: 0.0612 - acc: 0.9822
Epoch 9/10
1123/1123 [==============================] - 0s 256us/step - loss: 0.0791 - acc: 0.9768
Epoch 10/10
1123/1123 [==============================] - 0s 258us/step - loss: 0.0579 - acc: 0.9813
Training of node 44 complete


Training chain node  45
Shapes:
 X = (1123, 1046) 
 Y = (1123, 1)
Epoch 1/10
1123/1123 [==============================] - 3s 3ms/step - loss: 0.2276 - acc: 0.9234
Epoch 2/10
1123/1123 [==

1123/1123 [==============================] - 0s 285us/step - loss: 0.0184 - acc: 0.9964
Epoch 9/10
1123/1123 [==============================] - 0s 283us/step - loss: 0.0178 - acc: 0.9964
Epoch 10/10
1123/1123 [==============================] - 0s 263us/step - loss: 0.0172 - acc: 0.9973
Training of node 51 complete


Training chain node  52
Shapes:
 X = (1123, 1053) 
 Y = (1123, 1)
Epoch 1/10
1123/1123 [==============================] - 4s 3ms/step - loss: 0.1080 - acc: 0.9653
Epoch 2/10
1123/1123 [==============================] - 0s 270us/step - loss: 0.0172 - acc: 0.9991
Epoch 3/10
1123/1123 [==============================] - 0s 278us/step - loss: 0.0156 - acc: 0.9991
Epoch 4/10
1123/1123 [==============================] - 0s 274us/step - loss: 0.0151 - acc: 0.9991
Epoch 5/10
1123/1123 [==============================] - 0s 274us/step - loss: 0.0151 - acc: 0.9991
Epoch 6/10
1123/1123 [==============================] - 0s 271us/step - loss: 0.0151 - acc: 0.9991
Epoch 7/10
1123/1123 [==

In [10]:
#Model chain
y_pred = []

X_feed = X_test.copy()
i = 0
for node in chain:
    print("Getting op from chain node ",i)
    node_name = "Node" + str(i)
    print("Shapes:\n X = {}".format(X_feed.shape))
    output = node.predict(X_feed)
    output = output.round().astype(int)
    y_pred.append(output)
    X_feed = np.append(X_feed,output,axis=1)
    i+=1
    #Updating X_feed
            

Getting op from chain node  0
Shapes:
 X = (579, 1001)
Getting op from chain node  1
Shapes:
 X = (579, 1002)
Getting op from chain node  2
Shapes:
 X = (579, 1003)
Getting op from chain node  3
Shapes:
 X = (579, 1004)
Getting op from chain node  4
Shapes:
 X = (579, 1005)
Getting op from chain node  5
Shapes:
 X = (579, 1006)
Getting op from chain node  6
Shapes:
 X = (579, 1007)
Getting op from chain node  7
Shapes:
 X = (579, 1008)
Getting op from chain node  8
Shapes:
 X = (579, 1009)
Getting op from chain node  9
Shapes:
 X = (579, 1010)
Getting op from chain node  10
Shapes:
 X = (579, 1011)
Getting op from chain node  11
Shapes:
 X = (579, 1012)
Getting op from chain node  12
Shapes:
 X = (579, 1013)
Getting op from chain node  13
Shapes:
 X = (579, 1014)
Getting op from chain node  14
Shapes:
 X = (579, 1015)
Getting op from chain node  15
Shapes:
 X = (579, 1016)
Getting op from chain node  16
Shapes:
 X = (579, 1017)
Getting op from chain node  17
Shapes:
 X = (579, 1018)
Ge

In [11]:
predictions = np.array([0])
for i in y_pred:
    x = np.array(i)
    if predictions.shape == (1,):
        predictions = x
    else:
        predictions = np.append(predictions,x,axis=1)

In [12]:
from sklearn.metrics import accuracy_score,hamming_loss, log_loss, f1_score

In [13]:
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("Hamming accuracy = ",hamming_accuracy_score(y_test,predictions))
print("Hamming loss = ",hamming_loss(y_test,predictions))
print("Log loss = ",log_loss(y_test,predictions))
print("F1 score = ",f1_score(y_test,predictions,average='macro'))

Accuracy =  0.13298791019
Hamming accuracy =  0.448076367118
Hamming loss =  0.04888063349301007
Log loss =  56.0167525591
F1 score =  0.199686680276
